# Get Data

This notebook aims to retrieve documents and texts for the analysis. 
There are a lot of different types of documents on the Vatican Website. I focus mainly on the following categories: 

1. Angelus
2. Omelie

In this work i want to cover the last 140 years circa, from Papa Leone XIII to Francesco. Obviuosly there isn't the same amount of document for each Pope, the most of the documents retrieved were written in recent times. 

In [13]:
from bs4 import BeautifulSoup
import urllib.request
import pymongo
import unicodedata
import time

In [14]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["vatican"]
dataset = db['texts']

Firstly we have to retrieve the list of the Holy Faters e their own urls to the documents

In [17]:
#Pagina con la lista di tutti i papi
req = urllib.request.urlopen('https://www.vatican.va/content/vatican/it/holy-father.index.html')
print("[STATUS]", req.getcode())
holyFatherPage = BeautifulSoup(str(req.read()), 'html.parser')

holyFatherTable = holyFatherPage.find(id='holy-father').find_all('a')
docCounter = 0

for i, a in enumerate(holyFatherTable):
    if(i < 255):
        continue
    
    time.sleep(5)
    
    aCounter, hCounter, mCounter = 0, 0, 0
        
    #Pagina di sommario di ogni papa
    page = BeautifulSoup(str(urllib.request.urlopen('https://www.vatican.va/'+a.get('href')).read()), 'html.parser')
    site = page.find("a", {'title': 'Sito Web'})
    name = page.find("header").find("h1").get_text().replace('\\n', '').replace('\\t', '')
    
    if(not site): continue 
    print(name)
    
    #Sito web dei documenti di ogni papa
    popePage = BeautifulSoup(str(urllib.request.urlopen(site.get('href')).read()), 'html.parser')
    contentList = popePage.find(id = "accordionmenu")
    
    #Angelus - Regina Coeli
    angelusIndex = (contentList.find("ul", {'path': 'angelus'}))
    if(angelusIndex):
        #Trovo i link di ogni anno e li ciclo
        angelusIndexList = angelusIndex.find_all('a')
        for a in angelusIndexList:
            url = a.get('href')
            while(True):
                req = urllib.request.urlopen('https://www.vatican.va'+url)
                angelusIndexPage = BeautifulSoup(str(req.read()), 'html.parser')
                
                #Trovo i documenti
                angelusDocList = angelusIndexPage.find('div', {'class': 'documento'}).find('ul').find_all('h1')
                for b in angelusDocList: 
                    if(b.find('a')): 
                        time.sleep(1.5)
                        
                        req = urllib.request.urlopen('https://www.vatican.va'+(b.find('a').get('href')))
                        angelusPage = BeautifulSoup(str(req.read()), 'html.parser')

                        print("[",req.getcode(),"]", b.find('a').get('href'))
                        
                        try:
                            text = angelusPage.find('div', {'class': 'testo'})\
                                .get_text()\
                                .replace('\\n', '').replace('\\t', '').replace('\\r', '')
                        except: 
                            print("[ERRORE] ", b.find('a').get('href'))
                        
                        item = {'author': name, 'type': 'Angelus', 
                                'url': b.find('a').get('href'), 
                                'text': unicodedata.normalize('NFKD', text)}
                        
                        try:
                            dataset.insert_one(item)
                            aCounter += 1
                        except: 
                            print("[ERRORE] Errore nell'inserimento del documento nel DB")
                        
                #Pagina successiva
                nextPage = angelusIndexPage.find("nav", {"class": "navigation-pages"})
                if(nextPage):
                    nextPage = nextPage.find("a", {"title": "Avanti"})
                    if(nextPage):
                        url = nextPage.get('href')
                    else:
                        break
                else:
                    break
    
    time.sleep(5)
    
    #Omelie
    homiliesIndex = (contentList.find("ul", {'path': 'homilies'}))
    if(homiliesIndex):
        #Trovo i link di ogni anno e li ciclo
        homiliesIndexList = homiliesIndex.find_all('a')
        for a in homiliesIndexList:
            url = a.get('href')
            while(True):
                req = urllib.request.urlopen('https://www.vatican.va'+url)
                homiliesIndexPage = BeautifulSoup(str(req.read()), 'html.parser')
                
                #Trovo i documenti
                homiliesDocList = homiliesIndexPage.find('div', {'class': 'documento'}).find('ul').find_all('h1')
                for b in homiliesDocList: 
                    if(b.find('a')):
                        time.sleep(3)
                        
                        req = urllib.request.urlopen('https://www.vatican.va'+(b.find('a').get('href')))
                        homiliesPage = BeautifulSoup(str(req.read()), 'html.parser')

                        print("[",req.getcode(),"]", b.find('a').get('href'))                          
                                                     
                        try:
                            text = homiliesPage.find('div', {'class': 'testo'})\
                                .get_text()\
                                .replace('\\n', '').replace('\\t', '').replace('\\r', '')
                        except: 
                            print("[ERRORE] ", b.find('a').get('href'))
                        
                        item = {'author': name, 'type': 'Homilies', 
                                'url': b.find('a').get('href'), 
                                'text': unicodedata.normalize('NFKD', text)}
                        
                        try:
                            dataset.insert_one(item)
                            hCounter += 1
                        except: 
                            print("[ERRORE] Errore nell'inserimento del documento nel DB")
                    
                #Pagina successiva
                nextPage = homiliesIndexPage.find("nav", {"class": "navigation-pages"})
                if(nextPage):
                    nextPage = nextPage.find("a", {"title": "Avanti"})
                    if(nextPage): 
                        url = nextPage.get('href')
                    else:
                        break
                else:
                    break
    
    
    time.sleep(5)
    
    #Meditazioni Quotidiane
    medIndex = (contentList.find("ul", {'path': 'cotidie'}))
    if(medIndex):
        #Trovo i link di ogni anno e li ciclo
        medIndexList = medIndex.find_all('a')
        for a in medIndexList:
            url = a.get('href')
            while(True):
                req = urllib.request.urlopen('https://www.vatican.va'+url)
                medIndexPage = BeautifulSoup(str(req.read()), 'html.parser')
                
                #Trovo i documenti
                medDocList = medIndexPage.find('div', {'class': 'documento'}).find('ul').find_all('h1')
                for b in medDocList: 
                    if(b.find('a')):
                        time.sleep(3)
                        
                        req = urllib.request.urlopen('https://www.vatican.va'+(b.find('a').get('href')))
                        medPage = BeautifulSoup(str(req.read()), 'html.parser')

                        print("[",req.getcode(),"]", b.find('a').get('href'))                          
                                                     
                        try:
                            text = medPage.find('div', {'class': 'testo'})\
                                .get_text()\
                                .replace('\\n', '').replace('\\t', '').replace('\\r', '')
                        except: 
                            print("[ERRORE] ", b.find('a').get('href'))
                        
                        item = {'author': name, 'type': 'Meditazioni', 
                                'url': b.find('a').get('href'), 
                                'text': unicodedata.normalize('NFKD', text)}
                        
                        try:
                            dataset.insert_one(item)
                            mCounter += 1
                        except: 
                            print("[ERRORE] Errore nell'inserimento del documento nel DB")
                            
                #Pagina successiva
                nextPage = homiliesIndexPage.find("nav", {"class": "navigation-pages"})
                if(nextPage):
                    nextPage = nextPage.find("a", {"title": "Avanti"})
                    if(nextPage): 
                        url = nextPage.get('href')
                    else:
                        break
                else:
                    break
    
    print(aCounter, "Angelus collected", hCounter, "Homilies collected", mCounter, "Meditazioni collected")

    docCounter =+ (aCounter + hCounter + mCounter)

[STATUS] 200
Leone XIII
0 Angelus collected 0 Homilies collected 0 Meditazioni collected
Pio X
0 Angelus collected 0 Homilies collected 0 Meditazioni collected
Benedetto XV
0 Angelus collected 0 Homilies collected 0 Meditazioni collected
Pio XI
0 Angelus collected 0 Homilies collected 0 Meditazioni collected
Pio XII
0 Angelus collected 0 Homilies collected 0 Meditazioni collected
Giovanni XXIII
[ 200 ] /content/john-xxiii/it/homilies/1958/documents/hf_j-xxiii_hom_19581208_bergamo.html
[ 200 ] /content/john-xxiii/it/homilies/1958/documents/hf_j-xxiii_hom_19581123_primo-rito.html
[ 200 ] /content/john-xxiii/it/homilies/1959/documents/hf_j-xxiii_hom_19591104.html
[ 200 ] /content/john-xxiii/it/homilies/1959/documents/hf_j-xxiii_hom_19591011.html
[ 200 ] /content/john-xxiii/it/homilies/1959/documents/hf_j-xxiii_hom_19590628.html
[ 200 ] /content/john-xxiii/it/homilies/1959/documents/hf_j-xxiii_hom_19590607.html
[ 200 ] /content/john-xxiii/it/homilies/1959/documents/hf_j-xxiii_hom_19590517.

[ 200 ] /content/paul-vi/it/angelus/1965/documents/hf_p-vi_ang_19650926.html
[ 200 ] /content/paul-vi/it/angelus/1965/documents/hf_p-vi_ang_19091965.html
[ 200 ] /content/paul-vi/it/angelus/1965/documents/hf_p-vi_ang_12091965.html
[ 200 ] /content/paul-vi/it/angelus/1965/documents/hf_p-vi_ang_05091965.html
[ 200 ] /content/paul-vi/it/angelus/1965/documents/hf_p-vi_ang_19650829.html
[ 200 ] /content/paul-vi/it/angelus/1965/documents/hf_p-vi_ang_22081965.html
[ 200 ] /content/paul-vi/it/angelus/1965/documents/hf_p-vi_ang_15081965.html
[ 200 ] /content/paul-vi/it/angelus/1965/documents/hf_p-vi_ang_08081965.html
[ 200 ] /content/paul-vi/it/angelus/1965/documents/hf_p-vi_ang_01081965.html
[ 200 ] /content/paul-vi/it/angelus/1965/documents/hf_p-vi_ang_25071965.html
[ 200 ] /content/paul-vi/it/angelus/1965/documents/hf_p-vi_ang_18071965.html
[ 200 ] /content/paul-vi/it/angelus/1965/documents/hf_p-vi_ang_19650629.html
[ 200 ] /content/paul-vi/it/angelus/1965/documents/hf_p-vi_ang_19650627.html

[ 200 ] /content/paul-vi/it/angelus/1967/documents/hf_p-vi_ang_19670827.html
[ 200 ] /content/paul-vi/it/angelus/1967/documents/hf_p-vi_ang_19670820.html
[ 200 ] /content/paul-vi/it/angelus/1967/documents/hf_p-vi_ang_19670917.html
[ 200 ] /content/paul-vi/it/angelus/1967/documents/hf_p-vi_ang_19670815.html
[ 200 ] /content/paul-vi/it/angelus/1967/documents/hf_p-vi_ang_19670813.html
[ 200 ] /content/paul-vi/it/angelus/1967/documents/hf_p-vi_ang_19670806.html
[ 200 ] /content/paul-vi/it/angelus/1967/documents/hf_p-vi_ang_19670723.html
[ 200 ] /content/paul-vi/it/angelus/1967/documents/hf_p-vi_ang_19670716.html
[ 200 ] /content/paul-vi/it/angelus/1967/documents/hf_p-vi_ang_19670709.html
[ 200 ] /content/paul-vi/it/angelus/1967/documents/hf_p-vi_ang_19670702.html
[ 200 ] /content/paul-vi/it/angelus/1967/documents/hf_p-vi_ang_19670629.html
[ 200 ] /content/paul-vi/it/angelus/1967/documents/hf_p-vi_ang_19670625.html
[ 200 ] /content/paul-vi/it/angelus/1967/documents/hf_p-vi_ang_19670618.html

[ 200 ] /content/paul-vi/it/angelus/1969/documents/hf_p-vi_ang_19691026.html
[ 200 ] /content/paul-vi/it/angelus/1969/documents/hf_p-vi_ang_19691020.html
[ 200 ] /content/paul-vi/it/angelus/1969/documents/hf_p-vi_ang_19691012.html
[ 200 ] /content/paul-vi/it/angelus/1969/documents/hf_p-vi_ang_19691005.html
[ 200 ] /content/paul-vi/it/angelus/1969/documents/hf_p-vi_ang_19690928.html
[ 200 ] /content/paul-vi/it/angelus/1969/documents/hf_p-vi_ang_19690921.html
[ 200 ] /content/paul-vi/it/angelus/1969/documents/hf_p-vi_ang_19690914.html
[ 200 ] /content/paul-vi/it/angelus/1969/documents/hf_p-vi_ang_19690907.html
[ 200 ] /content/paul-vi/it/angelus/1969/documents/hf_p-vi_ang_19690831.html
[ 200 ] /content/paul-vi/it/angelus/1969/documents/hf_p-vi_ang_19690824.html
[ 200 ] /content/paul-vi/it/angelus/1969/documents/hf_p-vi_ang_19690815.html
[ 200 ] /content/paul-vi/it/angelus/1969/documents/hf_p-vi_ang_19690810.html
[ 200 ] /content/paul-vi/it/angelus/1969/documents/hf_p-vi_ang_19690803.html

[ 200 ] /content/paul-vi/it/angelus/1971/documents/hf_p-vi_ang_19711205.html
[ 200 ] /content/paul-vi/it/angelus/1971/documents/hf_p-vi_ang_19711128.html
[ 200 ] /content/paul-vi/it/angelus/1971/documents/hf_p-vi_ang_19711121.html
[ 200 ] /content/paul-vi/it/angelus/1971/documents/hf_p-vi_ang_19711114.html
[ 200 ] /content/paul-vi/it/angelus/1971/documents/hf_p-vi_ang_19711107.html
[ 200 ] /content/paul-vi/it/angelus/1971/documents/hf_p-vi_ang_19711101.html
[ 200 ] /content/paul-vi/it/angelus/1971/documents/hf_p-vi_ang_19711031.html
[ 200 ] /content/paul-vi/it/angelus/1971/documents/hf_p-vi_ang_19711024.html
[ 200 ] /content/paul-vi/it/angelus/1971/documents/hf_p-vi_ang_19711017.html
[ 200 ] /content/paul-vi/it/angelus/1971/documents/hf_p-vi_ang_19711010.html
[ 200 ] /content/paul-vi/it/angelus/1971/documents/hf_p-vi_ang_19711003.html
[ 200 ] /content/paul-vi/it/angelus/1971/documents/hf_p-vi_ang_19710926.html
[ 200 ] /content/paul-vi/it/angelus/1971/documents/hf_p-vi_ang_19710919.html

...

[ 200 ] /content/francesco/it/homilies/2016/documents/papa-francesco_20160319_omelia-ordinazioni-episcopali.html
[ 200 ] /content/francesco/it/homilies/2016/documents/papa-francesco_20160304_omelia-liturgia-penitenza.html
[ 200 ] /content/francesco/it/homilies/2016/documents/papa-francesco_20160222_giubileo-omelia-curia-romana.html
[ 200 ] /content/francesco/it/homilies/2016/documents/papa-francesco_20160217_omelia-messico-ciudad-jaurez.html
[ 200 ] /content/francesco/it/homilies/2016/documents/papa-francesco_20160216_omelia-messico-religiosi.html
[ 200 ] /content/francesco/it/homilies/2016/documents/papa-francesco_20160215_omelia-messico-chiapas.html
[ 200 ] /content/francesco/it/homilies/2016/documents/papa-francesco_20160214_omelia-messico-ecatepec.html
[ 200 ] /content/francesco/it/homilies/2016/documents/papa-francesco_20160213_omelia-messico-guadalupe.html
[ 200 ] /content/francesco/it/homilies/2016/documents/papa-francesco_20160210_giubileo-omelia-invio-missionari-misericordia.h

[ 200 ] /content/francesco/it/homilies/2015/documents/papa-francesco_20150125_vespri-conversione-san-paolo.html
[ 200 ] /content/francesco/it/homilies/2015/documents/papa-francesco_20150118_srilanka-filippine-omelia-rizal-park.html
[ 200 ] /content/francesco/it/homilies/2015/documents/papa-francesco_20150117_srilanka-filippine-omelia-tacloban.html
[ 200 ] /content/francesco/it/homilies/2015/documents/papa-francesco_20150116_srilanka-filippine-omelia-cattedrale-manila.html
[ 200 ] /content/francesco/it/homilies/2015/documents/papa-francesco_20150114_srilanka-filippine-omelia-canonizzazione.html
[ 200 ] /content/francesco/it/homilies/2015/documents/papa-francesco_20150111_omelia-battesimo-signore.html
[ 200 ] /content/francesco/it/homilies/2015/documents/papa-francesco_20150106_omelia-epifania.html
[ 200 ] /content/francesco/it/homilies/2015/documents/papa-francesco_20150101_omelia-giornata-mondiale-pace.html
[ 200 ] /content/francesco/it/homilies/2014/documents/papa-francesco_20141231_t

[ 200 ] /content/francesco/it/homilies/2013/documents/papa-francesco_20130907_veglia-pace.html
[ 200 ] /content/francesco/it/homilies/2013/documents/papa-francesco_20130828_capitolo-sant-agostino.html
[ 200 ] /content/francesco/it/homilies/2013/documents/papa-francesco_20130815_omelia-assunzione.html
[ 200 ] /content/francesco/it/homilies/2013/documents/papa-francesco_20130731_omelia-sant-ignazio.html
[ 200 ] /content/francesco/it/homilies/2013/documents/papa-francesco_20130728_celebrazione-xxviii-gmg.html
[ 200 ] /content/francesco/it/homilies/2013/documents/papa-francesco_20130727_gmg-omelia-rio-clero.html
[ 200 ] /content/francesco/it/homilies/2013/documents/papa-francesco_20130724_gmg-omelia-aparecida.html
[ 200 ] /content/francesco/it/homilies/2013/documents/papa-francesco_20130708_omelia-lampedusa.html
[ 200 ] /content/francesco/it/homilies/2013/documents/papa-francesco_20130707_omelia-seminaristi-novizie.html
[ 200 ] /content/francesco/it/homilies/2013/documents/papa-francesco_2

[ 200 ] /content/francesco/it/cotidie/2018/documents/papa-francesco-cotidie_20181203_santa-marta.html
[ 200 ] /content/francesco/it/cotidie/2018/documents/papa-francesco-cotidie_20181130_santa-marta.html
[ 200 ] /content/francesco/it/cotidie/2018/documents/papa-francesco-cotidie_20181129_santa-marta.html
[ 200 ] /content/francesco/it/cotidie/2018/documents/papa-francesco-cotidie_20181127_santa-marta.html
[ 200 ] /content/francesco/it/cotidie/2018/documents/papa-francesco-cotidie_20181126_santa-marta.html
[ 200 ] /content/francesco/it/cotidie/2018/documents/papa-francesco-cotidie_20181115_santa-marta.html
[ 200 ] /content/francesco/it/cotidie/2018/documents/papa-francesco-cotidie_20181112_santa-marta.html
[ 200 ] /content/francesco/it/cotidie/2018/documents/papa-francesco-cotidie_20181109_santa-marta.html
[ 200 ] /content/francesco/it/cotidie/2018/documents/papa-francesco-cotidie_20181108_santa-marta.html
[ 200 ] /content/francesco/it/cotidie/2018/documents/papa-francesco-cotidie_201811

[ 200 ] /content/francesco/it/cotidie/2014/documents/papa-francesco-cotidie_20141218_storia-siamo-noi.html
[ 200 ] /content/francesco/it/cotidie/2014/documents/papa-francesco-cotidie_20141216_quelli-che-andranno-prima.html
[ 200 ] /content/francesco/it/cotidie/2014/documents/papa-francesco-cotidie_20141215_cuori-di-tenebra.html
[ 200 ] /content/francesco/it/cotidie/2014/documents/papa-francesco-cotidie_20141211_la-ninnananna-di-dio.html
[ 200 ] /content/francesco/it/cotidie/2014/documents/papa-francesco-cotidie_20141209_uscire-per-dare-la-vita.html
[ 200 ] /content/francesco/it/cotidie/2014/documents/papa-francesco-cotidie_20141204_senza-trucco-sulla-roccia.html
[ 200 ] /content/francesco/it/cotidie/2014/documents/papa-francesco-cotidie_20141202_solo-umile-comprende.html
[ 200 ] /content/francesco/it/cotidie/2014/documents/papa-francesco-cotidie_20141127_depressione-o-speranza.html
[ 200 ] /content/francesco/it/cotidie/2014/documents/papa-francesco-cotidie_20141124_da-dove-arriva-la-lu

In [18]:
dataset.count()

<ipython-input-18-e6e2b92da74f>:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  dataset.count()


6903

In [11]:
dataset.drop()